## Main

In [ ]:
import gc
import copy
import math
import random
import numpy as np
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from typing import Dict, Any, Tuple
import optuna
from optuna.pruners import SuccessiveHalvingPruner, HyperbandPruner
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from modules.labeling_lib import get_prices
from modules.labeling_lib import get_features
from modules.labeling_lib import get_labels_one_direction
from modules.labeling_lib import sliding_window_clustering
from modules.tester_lib import test_model_one_direction_clustering
from modules.export_lib import export_model_to_ONNX
import warnings
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

def fit_final_models(clustered: pd.DataFrame,
                     meta: pd.DataFrame,
                     oos_data: pd.DataFrame,
                     backward_data: pd.DataFrame,
                     hp: Dict[str, Any]) -> Tuple[float, float, Any, Any]:
    """
    Entrena modelo principal + meta‑modelo y evalúa en OOS y backward.

    Devuelve (R2_forward, R2_backward, model_main, meta_model).
    """
    # ---------- 1) main model_main ----------
    X_main = clustered.loc[:, ~clustered.columns.isin(['labels'] + list(meta.columns[meta.columns.str.contains('_meta_feature')]))]
    y_main = clustered['labels'].astype('int16')

    # ---------- 2) meta‑modelo ----------
    X_meta = meta.loc[:, meta.columns.str.contains('_meta_feature')]
    y_meta = meta['clusters'].astype('int16')
    
    # 3) Split aleatorio (70/30)
    train_X, test_X, train_y, test_y = train_test_split(
        X_main, y_main, train_size=0.7, shuffle=True, )
    train_X_m, test_X_m, train_y_m, test_y_m = train_test_split(
        X_meta, y_meta, train_size=0.7, shuffle=True)

    # 4) Hiper‑parámetros CatBoost (con valores por defecto + overrides)
    cat_main_params = dict(
        iterations=hp.get('cat_main_iterations', 500),
        depth=hp.get('cat_main_depth', 6),
        learning_rate=hp.get('cat_main_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_main_l2_leaf_reg', 3.0),
        early_stopping_rounds=hp.get('cat_main_early_stopping'),
        #rsm=hp.get('cat_main_rsm', 0.5),
        custom_loss=['Accuracy'],
        eval_metric='Accuracy',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    model_main = CatBoostClassifier(**cat_main_params)
    model_main.fit(train_X, train_y, eval_set=(test_X, test_y))

    cat_meta_params = dict(
        iterations=hp.get('cat_meta_iterations', 500),
        depth=hp.get('cat_meta_depth', 6),
        learning_rate=hp.get('cat_meta_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_meta_l2_leaf_reg', 3.0),
        early_stopping_rounds=hp.get('cat_meta_early_stopping'),
        custom_loss=['F1'],
        eval_metric='F1',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    meta_model = CatBoostClassifier(**cat_meta_params)
    meta_model.fit(train_X_m, train_y_m, eval_set=(test_X_m, test_y_m))

    # 5) Evaluación en datos fuera de muestra (forward)
    R2_forward = test_model_one_direction_clustering(
        oos_data,
        [model_main, meta_model],
        hp['full_forward'],
        hp['forward'],
        hp['markup'],
        hp['direction'],
        plt=False,
    )
    
    # 6) Evaluación en datos históricos (backward)
    R2_backward = test_model_one_direction_clustering(
        backward_data,
        [model_main, meta_model],
        hp['forward'],
        hp['backward'],
        hp['markup'],
        hp['direction'],
        plt=False,
    )
    
    # Validación menos estricta de scores mínimos
    if R2_forward == -1.0 or R2_backward == -1.0:
        return -10.0, -10.0, model_main, meta_model
        
    return R2_forward, R2_backward, model_main, meta_model

def objective(trial: optuna.trial.Trial, base_hp: Dict[str, Any], study=None) -> float:
    # ─── cálculo combinado simple y simétrico ──────────────────────────
    def calc_score(fwd: float, bwd: float, eps: float = 1e-9) -> float:
        if (fwd is None or bwd is None or
            not np.isfinite(fwd) or not np.isfinite(bwd) or
            fwd <= 0 or bwd <= 0):
            return -1.0
        mean   = 0.5 * (fwd + bwd)
        delta  = abs(fwd - bwd) / max(abs(fwd), abs(bwd), eps)
        score  = mean * (1.0 - delta)
        return score

    hp = {k: copy.deepcopy(v) for k, v in base_hp.items() if k != 'base_df'}
    gc.collect()
    # µ··· Espacio de búsqueda optimizado ···µ
    # Parámetros de clustering más amplios para encontrar patrones más diversos
    hp['n_clusters'] = trial.suggest_int('n_clusters', 5, 50, step=5)  
    hp['window_size'] = trial.suggest_int('window_size', 50, 500, step=10)
    hp['vol_period'] = trial.suggest_int('vol_period', 5, 50, step=5)
    
    # Parámetros de etiquetado más agresivos
    hp['label_min'] = trial.suggest_int('label_min', 1, 5)
    hp['label_max'] = trial.suggest_int('label_max', hp['label_min']+5, 30)
    hp['markup'] = trial.suggest_float("markup", 0.1, 0.4)

    # CatBoost principal - Mayor capacidad de aprendizaje
    hp['cat_main_iterations'] = trial.suggest_int('cat_main_iterations', 300, 2000, step=100)
    hp['cat_main_depth'] = trial.suggest_int('cat_main_depth', 6, 12)
    hp['cat_main_learning_rate'] = trial.suggest_float('cat_main_learning_rate', 0.005, 0.4, log=True)
    hp['cat_main_l2_leaf_reg'] = trial.suggest_float('cat_main_l2_leaf_reg', 0.5, 10.0)
    hp['cat_main_early_stopping'] = trial.suggest_int('cat_main_early_stopping', 10, 50, step=10)
    #hp['cat_main_rsm'] = trial.suggest_float('cat_main_rsm', 0.1, 1.0)

    # CatBoost meta - Enfoque en precisión
    hp['cat_meta_iterations'] = trial.suggest_int('cat_meta_iterations', 200, 1000, step=100)
    hp['cat_meta_depth'] = trial.suggest_int('cat_meta_depth', 5, 10)
    hp['cat_meta_learning_rate'] = trial.suggest_float('cat_meta_learning_rate', 0.01, 0.3, log=True)
    hp['cat_meta_l2_leaf_reg'] = trial.suggest_float('cat_meta_l2_leaf_reg', 0.5, 8.0)
    hp['cat_meta_early_stopping'] = trial.suggest_int('cat_meta_early_stopping', 10, 50, step=10)

    # Optimización de períodos para el modelo principal
    n_periods_main = trial.suggest_int('n_periods_main', 5, 15)
    main_periods = []
    for i in range(n_periods_main):
        period_main = trial.suggest_int(f'period_main_{i}', 5, 200, log=True)
        main_periods.append(period_main)
    main_periods = sorted(list(set(main_periods)))
    hp['periods_main'] = main_periods

    # Optimización de períodos para el meta-modelo
    n_periods_meta = 1 #trial.suggest_int('n_periods_meta', 1, 2)
    meta_periods = []
    for i in range(n_periods_meta):
        period_meta = trial.suggest_int(f'period_meta_{i}', 3, 7)
        meta_periods.append(period_meta)
    meta_periods = sorted(list(set(meta_periods)))
    hp['periods_meta'] = meta_periods

    # Selección de estadísticas para el modelo principal
    main_stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew"
    ]

    n_main_stats = trial.suggest_int('n_main_stats', 1, 5)
    selected_main_stats = []
    for i in range(n_main_stats):
        stat = trial.suggest_categorical(f'main_stat_{i}', main_stat_choices)
        selected_main_stats.append(stat)
    selected_main_stats = list(set(selected_main_stats))
    if len(selected_main_stats) == 1 and ("fractal" in selected_main_stats or "hurst" in selected_main_stats or "kurt" in selected_main_stats):
        remaining_stats = [s for s in main_stat_choices if s != "fractal" and s != "hurst" and s != "kurt"]
        additional_stat = trial.suggest_categorical('additional_stat', remaining_stats)
        selected_main_stats.append(additional_stat)
    hp["stats_main"] = selected_main_stats
    #print(f"Main features seleccionadas: {hp['stats_main']}")

    # Selección de estadísticas para el meta-modelo
    meta_stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew"
    ]
    # Seleccionar estadísticas meta
    n_meta_stats = trial.suggest_int('n_meta_stats', 1, 2)
    selected_meta_stats = []
    for i in range(n_meta_stats):
        stat = trial.suggest_categorical(f'meta_stat_{i}', meta_stat_choices)
        selected_meta_stats.append(stat)
    selected_meta_stats = list(set(selected_meta_stats))
    hp["stats_meta"] = selected_meta_stats
    #print(f"Meta features seleccionadas: {hp['stats_meta']} | Periodo: {hp['periods_meta']}")

    # Dataset completo
    full_ds = get_features(base_hp['base_df'], hp)
    
    # Dividir en períodos de entrenamiento, backward testing y forward testing
    train_mask = (full_ds.index > hp['backward']) & (full_ds.index < hp['forward'])
    oos_mask = (full_ds.index >= hp['forward']) & (full_ds.index < hp['full_forward'])
    ds_train = full_ds.loc[train_mask]
    ds_oos = full_ds.loc[oos_mask]
    ds_train_val = ds_train.iloc[-len(ds_oos):]
    
    # Clustering con ventana deslizante
    assert ds_train.index.max() < hp['forward'], \
       "¡ds_train contiene fechas posteriores al límite forward!"
    assert ds_oos.index.min() >= hp['forward'], \
        "¡ds_oos incluye datos dentro del tramo backward!"
    data = sliding_window_clustering(
        ds_train,
        n_clusters=hp['n_clusters'],
        window_size=hp['window_size'],
        vol_period=hp['vol_period']
    )
    
    best_combined_score = -math.inf
    
    # Calcular umbral mínimo adaptativo basado en el tamaño del dataset
    # total_samples = len(data)
    # min_samples_percent = 0.02  # 2% del total de muestras como mínimo
    # min_samples_absolute = 200  # Mínimo absoluto
    # min_samples_required = max(min_samples_absolute, int(total_samples * min_samples_percent))
    
    # Evaluar clusters ordenados por tamaño
    cluster_sizes = data['clusters'].value_counts()
    for clust in cluster_sizes.index:
        clustered_data = data.loc[data['clusters'] == clust]

        clustered_data = get_labels_one_direction(
            clustered_data,
            markup=hp['markup'],
            min=hp['label_min'],
            max=hp['label_max'],
            direction=hp['direction'])

        clustered_data = clustered_data.drop(['close', 'clusters'], axis=1)
        meta_data = data.copy()
        meta_data['clusters'] = (meta_data['clusters'] == clust).astype(int)

        # ── descartar clusters problemáticos ────────────────────────────
        label_counts  = clustered_data['labels'].value_counts()
        meta_counts   = meta_data['clusters'].value_counts()
        # 1) mínimo de muestras totales
        #if len(clustered_data) < min_samples_required:
        #    continue
        # 2) labels: deben existir las 2 clases y ≥ 5 ejemplos cada una
        if len(label_counts) < 2 or (label_counts < 5).any():
            continue
        # 3) meta-labels: idem (clusters 0 / 1)
        if len(meta_counts) < 2 or (meta_counts < 5).any():
            continue

        # Evaluación en ambos períodos
        R2_forward, R2_backward, model_main, meta_model = fit_final_models(
            clustered_data,
            meta_data.drop(['close'], axis=1),
            ds_oos,
            ds_train_val,
            hp
        )

        # Calcular puntuación combinada (puedes ajustar los pesos según necesites)
        # reemplaza todo el bloque anterior por:
        score = calc_score(R2_forward, R2_backward)
        if score <= -1.0:
            continue  

        if score > best_combined_score:
            best_combined_score = score
            # Guardar información del trial actual
            trial.set_user_attr("forward_r2", R2_forward)
            trial.set_user_attr("backward_r2", R2_backward)
            trial.set_user_attr("combined_score", score)
            trial.set_user_attr("cluster_id", clust)
            trial.set_user_attr("stats_main", hp["stats_main"])
            trial.set_user_attr("stats_meta", hp["stats_meta"])
            trial.set_user_attr("periods_main", hp["periods_main"])
            trial.set_user_attr("periods_meta", hp["periods_meta"])
            # Guardar parámetros del trial actual (sin fechas)
            params_to_save = hp.copy()
            params_to_save.pop('backward', None)
            params_to_save.pop('forward', None)
            params_to_save.pop('full_forward', None)
            trial.set_user_attr("params", params_to_save)
            # Si existe el estudio, actualizar sus atributos
            if study is not None:
                current_best = study.user_attrs.get("best_combined_score", -math.inf)
                if score > current_best:
                    study.set_user_attr("best_params", params_to_save)
                    study.set_user_attr("best_metrics", {
                        "forward_r2": R2_forward,
                        "backward_r2": R2_backward,
                        "combined_score": score,
                        "cluster_id": clust,
                    })
                    study.set_user_attr("best_combined_score", score)
                    study.set_user_attr("best_models", [model_main, meta_model])
                    study.set_user_attr("best_stats_main", hp["stats_main"])
                    study.set_user_attr("best_stats_meta", hp["stats_meta"])
                    study.set_user_attr("best_periods_main", hp["periods_main"])
                    study.set_user_attr("best_periods_meta", hp["periods_meta"])
                    study.set_user_attr("best_trial_number", trial.number)
                    study.set_user_attr("best_df_sample", full_ds.sample(1))
    # Si no hay ningún cluster válido, devolver un valor negativo pero no infinito
    if best_combined_score == -math.inf:
        return -10.0

    # No aplicar penalización adicional por pocos clusters para mantener coherencia
    return best_combined_score

def optimize_and_export(base_hp: Dict[str, Any]):
    """Lanza Optuna, guarda el mejor modelo y lo exporta a ONNX."""
    def show_best_summary(study: optuna.study.Study, model_seed) -> None:
        """Muestra en un único cuadro el resultado óptimo del estudio."""
        # ── extraer métricas y modelos ───────────────────────────────────────
        m  = study.user_attrs.get("best_metrics", {})
        f2 = m.get("forward_r2",   float("nan"))
        b2 = m.get("backward_r2",  float("nan"))
        c2 = m.get("combined_score", float("nan"))
        best_trial = study.user_attrs.get("best_trial_number", "-")
        best_df_sample = study.user_attrs.get("best_df_sample", None)

        main_cls = meta_cls = "None"
        best_models = study.user_attrs.get("best_models")
        if best_models and len(best_models) == 2:
            main_cls = type(best_models[0]).__name__
            meta_cls = type(best_models[1]).__name__

        # ── cuadro de resumen ────────────────────────────────────────────────
        lines = [
            "┌" + "─" * 55 + "┐",
            f"│  MODELO {model_seed} TRIAL ÓPTIMO #{best_trial}",
            "├" + "─" * 55 + "┤",
            f"│  R² Forward : {f2:10.4f}  │  R² Backward : {b2:10.4f} │",
            f"│  Combined     : {c2:10.4f}                            │",
            "├" + "─" * 55 + "┤"
        ]
        print("\n".join(lines))

        #debug
        # if best_df_sample is not None:
        #     pd.set_option('display.max_columns', None)
        #     pd.set_option('display.width', None)
        #     pd.set_option('display.max_colwidth', None)
        #     display(best_df_sample)

    base_hp.update({
        'model_seed': random.randint(0, 10000000),
    })
    # Configurar el pruner inteligente
    # pruner = SuccessiveHalvingPruner(
    #     min_resource=1,
    #     reduction_factor=3,
    #     min_early_stopping_rate=0
    # )

    # Crear el estudio sin persistencia
    study = optuna.create_study(
        direction='maximize',
        pruner=HyperbandPruner(),
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=int(np.sqrt(base_hp['n_trials'])),
        )
    )
    
    study.optimize(lambda t: objective(t, base_hp, study),
                n_trials=base_hp['n_trials'],
                show_progress_bar=True)

    # ── verificación y cuadro ───────────────────────────────
    best_trial = study.best_trial
    assert study.user_attrs.get("best_trial_number") == best_trial.number, \
        "best_trial_number no coincide con study.best_trial.number"

    best_metric_saved = study.user_attrs.get("best_metrics", {}).get("combined_score")
    if best_metric_saved is not None:
        assert math.isclose(best_metric_saved, study.best_value, rel_tol=1e-9), \
            "combined_score guardado ≠ study.best_value"
    else:
        print("⚠️  Ningún trial produjo un score válido; modelos no guardados.")

    best_models = study.user_attrs.get("best_models")
    assert best_models and len(best_models) == 2 and all(best_models), \
        "best_models incorrecto"

    show_best_summary(study, base_hp['model_seed'])

    # ── exportación ─────────────────────────────────────────
    print("Exportando modelos ONNX…")
    export_params = base_hp.copy()
    export_params.update({
        "best_trial": study.user_attrs["best_trial_number"],
        "best_score": study.user_attrs["best_combined_score"],
        "best_periods_main": study.user_attrs["best_periods_main"],
        "best_periods_meta": study.user_attrs["best_periods_meta"],
        "best_stats_main"  : study.user_attrs["best_stats_main"],
        "best_stats_meta"  : study.user_attrs["best_stats_meta"],
        "best_models"      : study.user_attrs["best_models"],
    })
    export_model_to_ONNX(**export_params)
    # ── devolver métricas ─────────────────────────────────────────
    return {
        "forward_r2"   : study.user_attrs.get("best_metrics", {}).get("forward_r2",   float("nan")),
        "backward_r2"  : study.user_attrs.get("best_metrics", {}).get("backward_r2",  float("nan")),
        "combined_score": study.user_attrs.get("best_metrics", {}).get("combined_score", float("nan")),
    }

if __name__ == "__main__":
    base_hp: Dict[str, Any] = {
        'symbol': r'XAUUSD',
        'timeframe': 'H1',
        'direction': 'buy',
        'backward': datetime(2020, 2, 1),
        'forward': datetime(2024, 2, 1),
        'full_forward': datetime(2026, 2, 1),
        'model_seed': 0,
        'n_trials': 1000,
        'models_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/',
        'include_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Include/ajmtrz/include/Dmitrievsky',
        'history_path': r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/",
        'stats_main': [],
        'stats_meta': [],
        'best_models': [None, None],
        'markup': 0.20,
        'label_min'  : 1,
        'label_max'  : 15,
        'n_clusters': 30,
        'window_size': 350,
        'periods_main': [i for i in range(5, 300, 30)],
        'periods_meta': [5],
    }
    base_df = get_prices(base_hp)
    base_hp.update({
        'base_df': base_df,
    })
    # Para recopilar resultados globales de todos los modelos
    all_results = {}
    best_models = []
    model_range = range(0, 10)
    base_hp.update({
        'n_trials': 1000,
    })
    for i in tqdm(model_range, desc=f"Optimizando {base_hp['symbol']}/{base_hp['timeframe']}", unit="modelo"):
        try:
            model_results = optimize_and_export(base_hp)
            best_models.append((i, model_results))
            
            # Añadir a resultados globales
            all_results[f"model_{i}"] = {
                "success": True,
                "forward_r2": model_results["forward_r2"],
                "backward_r2": model_results["backward_r2"],
                "combined_score": model_results["combined_score"]
            }
            
        except Exception as e:
            import traceback
            tqdm.write(f"\nError procesando modelo {i}: {str(e)}")
            tqdm.write(traceback.format_exc())
            
            all_results[f"model_{i}"] = {
                "success": False,
                "error": str(e)
            }
            continue
    
    # Resumen final
    print("\n" + "="*50)
    print(f"RESUMEN DE OPTIMIZACIÓN {base_hp['symbol']}/{base_hp['timeframe']}")
    print("="*50)
    
    successful_models = [info for model_key, info  in all_results.items() if info.get("success", False)]
    print(f"Modelos completados exitosamente: {len(successful_models)}/{len(model_range)}")
    
    if successful_models:
        # Calcular estadísticas globales
        forward_scores = [info["forward_r2"] for info in successful_models]
        backward_scores = [info["backward_r2"] for info in successful_models]
        combined_scores = [info["combined_score"] for info in successful_models]
        
        print(f"\nEstadísticas de rendimiento:")
        print(f"  Forward R2 promedio: {np.mean(forward_scores):.4f} ± {np.std(forward_scores):.4f}")
        print(f"  Backward R2 promedio: {np.mean(backward_scores):.4f} ± {np.std(backward_scores):.4f}")
        print(f"  Puntuación combinada promedio: {np.mean(combined_scores):.4f} ± {np.std(combined_scores):.4f}")

        # Identificar el mejor modelo global basado en la puntuación combinada
        successful = [(k, v) for k, v in all_results.items() if v.get("success", False)]
        combined_scores = [v["combined_score"] for _, v in successful]
        best_model_key, best_info = successful[int(np.argmax(combined_scores))]
        
        print(f"\nMejor modelo global: {best_model_key}")
        print(f"  Forward R2: {best_info['forward_r2']:.4f}")
        print(f"  Backward R2: {best_info['backward_r2']:.4f}")
        print(f"  Puntuación combinada: {best_info['combined_score']:.4f}")
    
    print("\nProceso de optimización completado.")